[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2022/blob/main/50--spark-multi-source/11--mongo--movies.ipynb)



In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz -O spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

--2023-01-21 01:42:14--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299350810 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop3.tgz’

spark-3.3.1-bin-had 100%[===================>] 285.48M  85.5MB/s    in 3.4s    

2023-01-21 01:42:18 (84.3 MB/s) - ‘spark-3.3.1-bin-hadoop3.tgz’ saved [299350810/299350810]



In [2]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.5 MB/s eta 0:00:00


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [4]:
!sudo apt install mongodb
!sudo service mongodb start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb-clients mongodb-server
  mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb mongodb-clients mongodb-server
  mongodb-server-core
0 upgraded, 7 newly installed, 0 to remove and 23 not upgraded.
Need to get 55.8 MB of archives.
After this operation, 226 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcap0.8 amd64 1.9.1-3 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libyaml-cpp0.6 amd64 0.6.2-4ubuntu1 [124 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 mongo-tools amd64 3.6.3-0ubuntu1 [12.3 MB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 mongodb-clients amd64 1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5.3 [21.6 MB]
Get:5 

In [5]:

!java -version

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)


In [6]:
!mkdir -p var
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson -O var/video_movies.bson
!wget https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movieDetails.bson -O var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection movies var/video_movieDetails.bson
!mongorestore --db datascience --drop --collection films var/video_movies.bson

--2023-01-21 01:43:33--  https://github.com/SouthbankSoftware/dbkoda-data/raw/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson [following]
--2023-01-21 01:43:33--  https://raw.githubusercontent.com/SouthbankSoftware/dbkoda-data/master/SampleCollections/dump/SampleCollections/video_movies.bson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 343355 (335K) [application/octet-stream]
Saving to: ‘var/video_movies.bson’

var/video_movies.b

In [7]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://localhost:27017')
db = client.datascience

In [9]:
pp(list(db.movies.aggregate([
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
    {"$limit": 2}
])))

[{'_id': ObjectId('5692a53024de1e0ce2dfdca5'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 7.6,
  'ratio': 22.0,
  'title': 'Au bout du monde'},
 {'_id': ObjectId('5692a47c24de1e0ce2dfdb63'),
  'awards': {'nominations': 1, 'text': '22 wins & 1 nomination.', 'wins': 22},
  'rating': 8.1,
  'ratio': 22.0,
  'title': 'Everything Will Be Ok'}]


In [ ]:
!rm /content/spark-3.3.1-bin-hadoop3/jars/mongo*

In [14]:
!rm /content/spark-3.3.1-bin-hadoop3/jars/mongo*
!ls -la /content/spark-3.3.1-bin-hadoop3/jars | grep mongo

-rw-r--r--  1 root      root  2314847 Apr 22  2022 mongo-java-driver-3.12.11.jar
-rw-r--r--  1 root      root        0 Jan 21 01:44 mongo-spark-connector_2.13-10.0.5-all.jar


In [13]:

!wget https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.13/10.0.5/mongo-spark-connector_2.13-10.0.5-all.jar -O /content/spark-3.3.1-bin-hadoop3/jars/mongo-spark-connector_2.13-10.0.5-all.jar
!wget https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.11/mongo-java-driver-3.12.11.jar -O /content/spark-3.3.1-bin-hadoop3/jars/mongo-java-driver-3.12.11.jar

--2023-01-21 01:44:49--  https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.13/10.0.5/mongo-spark-connector_2.13-10.0.5-all.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-01-21 01:44:49 ERROR 404: Not Found.

--2023-01-21 01:44:49--  https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.11/mongo-java-driver-3.12.11.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2314847 (2.2M) [application/java-archive]
Saving to: ‘/content/spark-3.3.1-bin-hadoop3/jars/mongo-java-driver-3.12.11.jar’

/content/spark-3.3. 100%[===================>]   2.21M  --.-KB/s    in 0.05s   

2023-01-21 01:44:49 (48.6 MB/s) - ‘/content

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("local").\
        config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5,org.mongodb:mongo-java-driver:3.12.11').\
        config("spark.mongodb.input.uri","mongodb://localhost:27017/datascience.movies").\
        getOrCreate()

sc = spark.sparkContext

In [ ]:
dfAsItIs = (spark.read.format("mongodb")
  .option("spark.mongodb.read.connection.uri","mongodb://127.0.0.1/datascience.movies")
  .load())


In [ ]:
dfAsItIs.show()

In [16]:
print(sc._jsc.sc().listJars())

Vector(spark://d47f0e6e6ab0:33409/jars/org.mongodb_mongo-java-driver-3.12.11.jar, spark://d47f0e6e6ab0:33409/jars/org.mongodb_mongodb-driver-sync-4.5.1.jar, spark://d47f0e6e6ab0:33409/jars/org.mongodb.spark_mongo-spark-connector-10.0.5.jar, spark://d47f0e6e6ab0:33409/jars/org.mongodb_bson-4.5.1.jar, spark://d47f0e6e6ab0:33409/jars/org.mongodb_mongodb-driver-core-4.5.1.jar)


In [17]:
pipeline = [
    {"$match": {"awards": {"$exists": "1"}}},
    {"$match": {"awards.wins": {"$gt": 0}, "awards.nominations": {"$gt": 0}}},
    {
        "$project": {
            "title": 1,
            "awards": 1,
            'rating': '$imdb.rating',
            "ratio": {
                "$divide": ['$awards.wins', '$awards.nominations']
            }
        }
    },
    {"$sort": {"ratio": -1}},
]

In [18]:
import json

In [20]:
df = (spark.read.format("mongodb")
  .option("spark.mongodb.read.connection.uri","mongodb://127.0.0.1/datascience.movies")
  .option("aggregation.pipeline", json.dumps(pipeline))
  .load())

In [21]:
df.show()

+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+-------+-------------------+-------+--------------------+--------------------+-----+--------------------+------+
|                 _id|              actors|              awards|         countries|            director|              genres|                imdb|metacritic|                plot|              poster|  rated|           released|runtime|               title|              tomato| type|             writers|  year|
+--------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+-------+-------------------+-------+--------------------+--------------------+-----+--------------------+------+
|569190cb24de1e0ce...|[Seth MacFarlane,...|{6.0, null, 6 nom...|

In [22]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- actors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- awards: struct (nullable = true)
 |    |-- nominations: double (nullable = true)
 |    |-- oscars: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- bestAnimatedFeature: string (nullable = true)
 |    |    |    |-- bestMusic: string (nullable = true)
 |    |    |    |-- bestPicture: string (nullable = true)
 |    |    |    |-- bestScreenplay: string (nullable = true)
 |    |    |    |-- bestSoundEditing: string (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- wins: double (nullable = true)
 |-- countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- director: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- imdb: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- rating: do

In [ ]:
df = (spark.read.format("mongodb")
  .option("spark.mongodb.read.connection.uri","mongodb://127.0.0.1/datascience.movies")
  .load())

In [23]:
from pyspark.sql import functions as F

In [25]:
df.select(
    F.col('title'),
    F.col('awards.wins').alias('wins'),
    F.col('imdb.rating').alias('rating')
).show(5)

+--------------------+----+------+
|               title|wins|rating|
+--------------------+----+------+
|A Million Ways to...| 0.0|   6.1|
|     West of Memphis| 0.0|   7.9|
|Star Wars: Episod...|38.0|   8.7|
|Star Wars: Episod...|15.0|   8.4|
|Star Wars: Episod...|13.0|   6.7|
+--------------------+----+------+
only showing top 5 rows



In [27]:
df.createOrReplaceTempView('movies')

In [28]:
spark.sql("Select * from movies").show(3)

+--------------------+--------------------+--------------------+------------------+---------------+--------------------+--------------------+----------+--------------------+--------------------+-----+-------------------+-------+--------------------+--------------------+-----+--------------------+------+
|                 _id|              actors|              awards|         countries|       director|              genres|                imdb|metacritic|                plot|              poster|rated|           released|runtime|               title|              tomato| type|             writers|  year|
+--------------------+--------------------+--------------------+------------------+---------------+--------------------+--------------------+----------+--------------------+--------------------+-----+-------------------+-------+--------------------+--------------------+-----+--------------------+------+
|569190cb24de1e0ce...|[Seth MacFarlane,...|{6.0, null, 6 nom...|             [USA]|Se

In [31]:
spark.sql("Select title, awards.wins as wins, imdb.rating from movies").show(3)

+--------------------+----+------+
|               title|wins|rating|
+--------------------+----+------+
|A Million Ways to...| 0.0|   6.1|
|     West of Memphis| 0.0|   7.9|
|Star Wars: Episod...|38.0|   8.7|
+--------------------+----+------+
only showing top 3 rows



In [32]:
moviesDF = spark.sql("Select title, awards.wins as wins, imdb.rating from movies")

Save it to external storage

In [33]:
moviesDF.write.csv('var/movies')

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
moviesDF.write.csv('drive/MyDrive/My-movies')

In [37]:
spark.stop()